<a href="https://colab.research.google.com/github/redinbluesky/the-lm-book/blob/main/05_대규모_언어_모델.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차
* [Chapter 5_1 규모가 클수록 좋은 이유](#chapter5_1)
* [Chapter 5_1_1 대규모 파라미터 개수](#chapter5_1_1) 
* [Chapter 5_1_2 큰 문맥 크기](#chapter5_1_2) 
* [Chapter 5_1_3 대규모 훈련 데이터셋](#chapter5_1_3) 
* [Chapter 5_1_4 대량의 계산량](#chapter5_1_4) 


## Chapter 5_1 규모가 클수록 좋은 이유 <a class="anchor" id="chapter5_1"></a>
1. LLM은 파라미터가 많고, 문맥 윈도가 크로, 상당한 컴퓨팅 자원을 바탕으로 대규모 말뭉치에서 훈련된다.
    - 이런 규모 덕분에 복잡한 언어 패턴을 학습하고 심지어 정보를 기억할 수 있다.

2. 대화를 이어가고 복잡한 지시를 따를 수 있는 채팅 LM을 만드는 작업은 두 단계로 구성된다.

2. 1단계
    - 수조 개의 토큰으로 구성된 대규모 데이터셋에서 사전훈련 진행
        - 모델은 무맥을 바탕으로 다음 토큰을 예측하는 방법을 학습나다.
    - 다음과 같은 문맥이 있다고 가정한다.
        <pre>
        "The CRISPR-Cas9 technique has revolutionized genetic engineering by enabling precise modification of DNA sequences. 
        The process uses a guide RNA to target specific locations in the genome, allowing scientists to add, remove, or alter genetic material with unprecedented accuracy. 
        This technology has vast potential applications, including treating genetic disorders, improving crop resilience, and advancing our understanding of gene functions."
        </pre>
    - 다음 토큰을 정화하게 예측하려면 모델은 다음과 같은 내용을 알아야한다.
        - CRISPR-Cas9 기술이 무엇인지
        - 유전자 공학에서의 역할
        - 가이드 RNA의 기능
        - 유전체에서 특정 위치를 타겟팅하는 방법
        - 유전 물질을 추가, 제거 또는 변경하는 방법
        - 이 기술의 잠재적 응용 분야
    - 잘 훈련된 LLM은 "CRISPR-Cas9"과 같은 복잡한 개념을 이해하고, 관련 정보를 기억하며, 문맥에 맞는 다음 토큰을 생성할 수 있다.
    - 모델이 표면적인 패턴에 의지하지 않고 유전자 편집 과정에 대한 깊은 이해를 바탄으로 문맥을 임베딩 벡터로 인코딩해야한다.
    - GPT-3은 비교적 복잡한 패턴을 이어가는 능력을 보여주었고, GPT-3.5와 GPT-4는 더욱 향상된 성능을 보여주었다.

        

### Chapter 5_1_1 대규모 파라미터 개수 <a class="anchor" id="chapter5_1_1"></a>
1. LLM의 가장 놀라운 특징 중 하나는 엄청난 파라미터 개수이다.
    - 최신 LLM은 수정 억 개 또는 심지어 수조 개의 파라미터를 가질 수 있다.

2. 트랜스포모 모델에서 파라미터의 개수는 임배딩 차원과 디코더 블록 개수에 의해 크개 좌우된다.
    - 이런 값이 커질수록 셀프 어텐션과 MLP 층에서 사용하는 임베딩 차원의 제곱 비율로 파라미터 개수가 증가하고, 디코더 블록의 개수와 함께 선형적으로 증가한다.
    - 예를 들어, 임베딩 차원이 2048이고 디코더 블록 개수가 24인 트랜스포머 모델은 약 2억 5천만 개의 파라미터를 가진다.
    - 임베딩 차원이 12288이고 디코더 블록 개수가 96인 모델은 약 1천 70억 개의 파라미터를 가진다.

3. 이 책에서 만든 모델과 여러 오픈 웨이트 LLM의 측징을 다음과 같은 이미지에 확인할 수 있다.
    - 관례적으로 오픈 웨이트 모델의 이름에서 "B"는 10억 단위의 파라미터 개수를의미한다.
    - 70B 모델의 각 파라미터를 32비트 부동소수점으로 저장하려면 약 280GB의 메모리가 필요하다.
    
        - ![llm-parameters](image/04-00-llm-parameters.png)

### Chapter 5_1_2 큰 문맥 크기 <a class="anchor" id="chapter5_1_2"></a>
1. 최신 LLM은 수천 개의 토큰, 심지어 수백만 개의 토큰까지도 문맥으로 처리할 수 있다.
    - 예를 들어, GPT-4는 최대 128k 토큰의 문맥 크기를 지원한다.

2. 트랜스포머 코델에서 긴 텍스트를 처리 때 가장 큰 어려움은 셀프 어텐션 메커니즘의 복잗도 계산에 있다.
    - 길이가 n인 시퀸스의 경우 셀프 어텐션은 모든 토큰 쌍 사이의 어텐션 점수를 계산해야 하므로 시간 및 공간 복잡도는 2차 다항 복잡도에 해당하는 O(n^2)이다.
    - 예를 들어, 문맥 크기가 1,024 토큰인 경우 셀프 어텐션 매트릭스는 1,024 x 1,024 = 1,048,576개의 어텐션 점수를 계산해야 한다.

3.  구조를 개선하고 어텐션 계산을 최적화하여 문캑 크기를 늘릴 수 있다.
    - 그룹 쿼리 어테션(GQA)과 같은 기법은 어텐션 계산을 단순화하여 긴 시퀸스를 더 효율적으로 처리할 수 있다.
       - GQA는 쿼리 벡터를 그룹으로 묶어 각 그룹에 대해 단일 어텐션 점수를 계산한다.
    - 플래시 어텐션은 메모리 사용량을 줄이고 어텐션 계산 속도를 높이는 효율적인 알고리즘이다.
       - 플래시 어텐션은 어텐션 매트릭스를 한 번에 모두 계산하지 않고, 작은 청크로 나누어 처리하여 메모리 사용량을 줄인다.

4. 일반적으로 LLM은 4K ~ 8K개의 토큰 정도의 짧은 문맥에서 사전훈련된다.
    - 어텐션 메커니즘의 2차 다항도 복잡도로 인해 긴 시퀸스에서 훈련할 때 계산 비용이 급격히 증가하기 때문이다.

5. 긴 문맥을 다루는 능력은 초기 훈련 후에 이어진 특별 단계인 긴 문맥 사전훈련을 통해 나타나며 훈련 순서는 다음과 같다.
    - 긴 문맥을 위한 점진적 훈련
        - 모델의 문맥 윈도를 여러 단계를 거쳐 4,000 ~ 8,000개의 토큰에서 128,000 ~ 256,000개의 토큰으로 점진적으로 늘린다.
        - 문캑 길이를 증가시키고 모델이 두 가지 핵심 조건을 만족할 때까지 훈련한다.
            - 짧은 문맥에서 달성한 성능을 회복하고 유지할 수 있어야 한다.
            - "모래사장에서 바늘 찾기"평과와 같은 긴 문맥과 관련된 작업에서 성능이 향상되어야 한다.
                - "모래사장에서 바늘 찾기"평가는 긴 문맥에서 중요한 정보를 찾아내는 모델의 능력을 평가하는 벤치마크이다.
    - 셀프 어텐션을 위한 효율적인 규모 확장
        - 2차 계산 복잡도를 다루기 위해 문맥 병렬화(context parallelism)와 같은 기법을 사용하여 긴 시퀸스를 여러 GPU에 분산시켜 처리한다.
            - 문맥 병렬화는 긴 시퀸스를 여러 부분으로 나누어 각 부분을 별도의 GPU에서 처리하는 방법이다.
            - all-gather 연산을 사용하여 각 GPU에서 계산된 어텐션 점수를 결합한다.

### Chapter 5_1_3 대규모 훈련 데이터셋 <a class="anchor" id="chapter5_1_3"></a>
1. LLM의 성능을 뒷받짐하는 세 번째 인자는 훈련에 사용된 말뭉치 크기다.
    - 최신 LLM은 수조 개의 토큰으로 구성된 대규모 데이터셋에서 훈련된다.
    - 다양한 장르와 시대의 책과 문학
    - 다양한 주제의 웹 페이지와 온라인 글
    - 학숩 논문과 화학 연구자료
    - 소셜 미디어 게시물과 토론
    - 코드 저장소와 기술 문서

2. 아래의 그링음 공개 Dolma 데이터셋을 예를들어 LLM 훈련 데이터셋의 구성을 보여준다.

    - ![llm-dataset](image/04-00-llm-dataset.png)

3. 신경망 언어 모델이 이런 방대한 말뭉치에서 훈련하므로 일반적으로 딱 한 번 데이터를 처리한다.
    - 이를 단일 에포크 훈련이라고 한다.
    - 대규모 데이터셋에서 단일 에포크 훈련은 모델이 다양한 언어 패턴과 지식을 학습하는 데 충분하다.
    - 반면에 소규모 데이터셋에서는 여러 번 데이터를 반복해서 처리하는 다중 에포크 훈련이 필요하다.

### Chapter 5_1_4 대량의 계산량 <a class="anchor" id="chapter5_1_4"></a>
1. 한 대의 최신 GPU로 Dolma 데이터셋의 토큰 3조 개를 처리하려면 100년이 걸린다.
    - 따라서 LLM 훈련에는 수천 대의 GPU가 수주에서 수개월 동안 병렬로 작동해야 한다.
    - 예를 들어, GPT-3은 1,024개의 V100 GPU에서 약 한 달 동안 훈련되었다.
    - GPT-4는 수천 개의 A100 GPU에서 몇 주 동안 훈련되었다.

2. Llama-3.1의 훈련 과정은 4차원 병령화라 불리는 고급 기법을 사용하여 수천 대의 GPU에 걸쳐 계산을 효율적으로 분산시킨다.
    - 4차원 병렬화는 모델 파라미터, 배치 샘플, 시퀸스 길이, 어텐션 헤드의 네 가지 차원에서 병렬화를 수행한다.
    - 이를 통해 각 GPU가 처리하는 데이터 양을 줄이고 통신 오버헤드를 최소화하여 훈련 속도를 크게 향상시킨다.
    - 가중치를 행렬을 여러 장치에 분할하는 텐서 병렬화
    - 각기 다른 트랜스포커  층을 여러 GPU에 분산 시키 파이프라인 병렬화
    - 긴 시퀸스 처리를 위해 입력 시퀸스를 분할하는 문맥 병렬화
    - 여러 GPU에서 동시에 배치를 처리하고 사후 동기화를 수행하는 데이터 병렬화

3. 학습에 드는 자원에는 하드웨어, 전력, 냉방, 언지니어링 기술이 포함된다.

4. 오픈 웨이트 모델은 이런 장벽을 낮추고 있다.
    - 작은 조직은 지도 학습 미세 튜닝과 프롬트트 엔지니어링 과 같은 방법을 통해 기존의 모델을 활용할 수 있다.